# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
import utilities.helper
import trodes.read_exported

In [10]:
import shutil
from pathlib import Path


In [11]:
FONTSIZE = 20

In [12]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [13]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [14]:
LFP_SPECTRAL_DF = pd.read_pickle("../2024_06_03_spectral_preprocessing/proc/rce_pilot_2_10_per_trial_spectral_bans_sleap.pkl")

In [15]:
OUTCOME_TO_COLOR = {"both_rewarded": "green", 
"novel_lose": "#e67073",
"novel_win": "#93a5da",
"lose": "#951a1d",
"alone_rewarded": "#0499af",
"win": "#3853a3",
"omission": "orange"}

In [16]:
OUTCOME_TO_COLOR = {"both_rewarded": "green", 
"novel_lose": "#e67073",
                    "novel_lose_competitive": "#e67073",
                    "novel_lose_no_comp": "#e67073",
"novel_win": "#93a5da",
                    "novel_win_competitive": "#93a5da",
                    "novel_win_no_comp": "#93a5da",
"lose": "#951a1d",
                    "lose_no_comp": "#951a1d",
                    "lose_competitive": "#951a1d",
"alone_rewarded": "#0499af",
"win": "#3853a3",
                    "win_competitive": "#3853a3",
                    "win_no_comp": "#3853a3",

"omission": "orange"}

In [17]:
session_types = ["training", "omission", "both", "novel"]

In [18]:
BAND_TO_FREQ_PLOT = {'theta': (4, 12), 'gamma': (30, 50)}
BAND_TO_FREQ_COLOR = {'theta': "red", 'beta': "blue", 'gamma': "green"}

In [19]:
preprocessing_type = ""

In [20]:
comp_closeness_dict = {'Subj 1 blocking Subj 2': "competitive",
'Subj 2 Only': "no_comp",
'Subj 2 blocking Subj 1': "competitive",
'Subj 1 then Subj 2': "competitive", 
'Subj 1 Only': "no_comp",
'Subj 2 then Subj 1': "competitive",
'Close Call': "competitive",
'After trial': "no_comp"}

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [21]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [22]:
# Define a function to horizontally stack arrays
def stack_arrays(arrays):
    return np.vstack(arrays)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [23]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [24]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

index
tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
baseline_start_timestamp
baseline_stop_timestamp
baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity
baseline_video_frame
baseline_video_timestamps
trial_agent_location

In [25]:
LFP_SPECTRAL_DF

,index,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,...,baseline_BLA_RMS_filtered_power_all_frequencies,baseline_vHPC_RMS_filtered_power_all_frequencies,baseline_power_timestamps,trial_mPFC_RMS_filtered_power_all_frequencies,trial_MD_RMS_filtered_power_all_frequencies,trial_LH_RMS_filtered_power_all_frequencies,trial_BLA_RMS_filtered_power_all_frequencies,trial_vHPC_RMS_filtered_power_all_frequencies,trial_power_timestamps,in_video
0,0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,...,"[[0.0053445, 0.009117814, 0.013600164, 0.01733...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[530000, 540000, 550000, 560000, 570000, 58000...","[[0.0036949348, 0.007999057, 0.007659001, 0.00...","[[0.00042359624, 0.0016437627, 0.003002788, 0....","[[0.0011694891, 0.0023001411, 0.003132459, 0.0...","[[0.013438185, 0.007965831, 0.009173761, 0.006...","[[0.010709718, 0.013590651, 0.022826178, 0.034...","[1130000, 1140000, 1150000, 1160000, 1170000, ...",True
1,1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,...,"[[0.00074698013, 0.0019324338, 0.005081963, 0....","[[0.003017959, 0.004722126, 0.008384243, 0.007...","[2930000, 2940000, 2950000, 2960000, 2970000, ...","[[0.017848626, 0.008553031, 0.013145922, 0.013...","[[0.0011482978, 0.0012844221, 0.0074498164, 0....","[[0.0022857906, 0.0023879802, 0.009314143, 0.0...","[[0.0095668975, 0.005058259, 0.00864778, 0.009...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[3530000, 3540000, 3550000, 3560000, 3570000, ...",True
2,2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,...,"[[0.016759386, 0.01684738, 0.021563489, 0.0619...","[[0.030797603, 0.04075655, 0.0501173, 0.065892...","[5230000, 5240000, 5250000, 5260000, 5270000, ...","[[0.018773032, 0.01531122, 0.023816012, 0.0154...","[[0.005590848, 0.0029627604, 0.010847095, 0.00...","[[0.0060438192, 0.0049293297, 0.0144101195, 0....","[[0.0006228732, 0.0015296962, 0.009809779, 0.0...","[[0.00064211205, 0.0052212942, 0.00709957, 0.0...","[5830000, 5840000, 5850000, 5860000, 5870000, ...",True
3,3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,...,"[[0.002649237, 0.0030132718, 0.009533786, 0.00...","[[0.018817268, 0.009095607, 0.008399841, 0.008...","[7030000, 7040000, 7050000, 7060000, 7070000, ...","[[0.00018646946, 0.006876981, 0.010109808, 0.0...","[[0.0037040254, 0.013202633, 0.018202007, 0.02...","[[0.004745133, 0.018028835, 0.02716619, 0.0303...","[[0.014945692, 0.009991203, 0.014374675, 0.019...","[[0.020167792, 0.028338362, 0.033197377, 0.035...","[7630000, 7640000, 7650000, 7660000, 7670000, ...",True
4,4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,...,"[[0.009458285, 0.0050086575, 0.02026654, 0.016...","[[0.015467933, 0.035288256, 0.044622492, 0.053...","[8130000, 8140000, 8150000, 8160000, 8170000, ...","[[0.006448407, 0.0046525886, 0.010715893, 0.01...","[[0.016451372, 0.010215264, 0.025078585, 0.018...","[[0.014854475, 0.008580038, 0.025409121, 0.017...","[[0.0016100197, 0.003484194, 0.008553746, 0.00...","[[0.0038173422, 0.008268023, 0.053486157, 0.04...","[8730000, 8740000, 8750000, 8760000, 8770000, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,1364,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,60714,60814,60914,2.2,...,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.0071711177, 0.014786493, 0.034027837, 0.06...","[60080000, 60090

In [26]:
LFP_SPECTRAL_DF["competition_closeness"] = LFP_SPECTRAL_DF["competition_closeness"].apply(lambda x: comp_closeness_dict.get(x, ""))

In [27]:
LFP_SPECTRAL_DF["competition_closeness"]

0      competitive
1          no_comp
2      competitive
3          no_comp
4          no_comp
          ...     
962        no_comp
963        no_comp
964        no_comp
965        no_comp
966        no_comp
Name: competition_closeness, Length: 967, dtype: object

# Getting trial and spectra average

In [28]:
gamma_frequencies = (30, 51)
theta_frequencies = (4, 13)

In [29]:
LFP_SPECTRAL_DF[[col for col in LFP_SPECTRAL_DF if "calculation_frequencies" in col]].iloc[0][0][gamma_frequencies[0]: gamma_frequencies[1]]

array([30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42.,
       43., 44., 45., 46., 47., 48., 49., 50.])

In [30]:
trial_columns = [col for col in LFP_SPECTRAL_DF if ("trial" in col or "baseline" in col) and "label" not in col and "timestamp" not in col and "frame" not in col and "locations" not in col]

In [31]:
trial_columns

['baseline_agent_thorax_to_reward_port',
 'baseline_agent_thorax_velocity',
 'baseline_subject_thorax_to_reward_port',
 'baseline_subject_thorax_velocity',
 'trial_agent_thorax_to_reward_port',
 'trial_agent_thorax_velocity',
 'trial_subject_thorax_to_reward_port',
 'trial_subject_thorax_velocity',
 'baseline_BLA_LH_RMS_filtered_coherence_all_frequencies',
 'baseline_BLA_MD_RMS_filtered_coherence_all_frequencies',
 'baseline_BLA_mPFC_RMS_filtered_coherence_all_frequencies',
 'baseline_BLA_vHPC_RMS_filtered_coherence_all_frequencies',
 'baseline_LH_MD_RMS_filtered_coherence_all_frequencies',
 'baseline_LH_mPFC_RMS_filtered_coherence_all_frequencies',
 'baseline_LH_vHPC_RMS_filtered_coherence_all_frequencies',
 'baseline_MD_mPFC_RMS_filtered_coherence_all_frequencies',
 'baseline_MD_vHPC_RMS_filtered_coherence_all_frequencies',
 'baseline_mPFC_vHPC_RMS_filtered_coherence_all_frequencies',
 'trial_BLA_LH_RMS_filtered_coherence_all_frequencies',
 'trial_BLA_MD_RMS_filtered_coherence_all_fr

In [32]:
for col in trial_columns:
    try:
        # For data that is 2 dimensional like the spectral analysis
        if len(LFP_SPECTRAL_DF[col].iloc[0].shape) == 2:
            # print(col)
            current_col = col.replace("RMS_filtered_", "").replace("_all_frequencies", "")
            LFP_SPECTRAL_DF["{}_theta".format(current_col)] = LFP_SPECTRAL_DF[col].apply(lambda x: np.nanmean(x[:, theta_frequencies[0]:theta_frequencies[1]]))
            LFP_SPECTRAL_DF["{}_gamma".format(current_col)] = LFP_SPECTRAL_DF[col].apply(lambda x: np.nanmean(x[:, gamma_frequencies[0]: gamma_frequencies[1]]))

            print(current_col)
        # For data that is 1 dimensional like SLEAP
        elif len(LFP_SPECTRAL_DF[col].iloc[0].shape) == 1:
            LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF[col].apply(lambda x: np.nanmean(x))
        else:
            print(col)
    except:
        pass


baseline_BLA_LH_coherence
baseline_BLA_MD_coherence
baseline_BLA_mPFC_coherence
baseline_BLA_vHPC_coherence
baseline_LH_MD_coherence
baseline_LH_mPFC_coherence
baseline_LH_vHPC_coherence
baseline_MD_mPFC_coherence
baseline_MD_vHPC_coherence
baseline_mPFC_vHPC_coherence
trial_BLA_LH_coherence
trial_BLA_MD_coherence
trial_BLA_mPFC_coherence
trial_BLA_vHPC_coherence
trial_LH_MD_coherence
trial_LH_mPFC_coherence
trial_LH_vHPC_coherence
trial_MD_mPFC_coherence
trial_MD_vHPC_coherence
trial_mPFC_vHPC_coherence
baseline_BLA_LH_granger
baseline_LH_BLA_granger
baseline_BLA_MD_granger
baseline_MD_BLA_granger
baseline_BLA_mPFC_granger
baseline_mPFC_BLA_granger
baseline_BLA_vHPC_granger
baseline_vHPC_BLA_granger
baseline_LH_MD_granger
baseline_MD_LH_granger
baseline_LH_mPFC_granger
baseline_mPFC_LH_granger
baseline_LH_vHPC_granger
baseline_vHPC_LH_granger
baseline_MD_mPFC_granger
baseline_mPFC_MD_granger
baseline_MD_vHPC_granger
baseline_vHPC_MD_granger
baseline_mPFC_vHPC_granger
baseline_vHPC_mPF

In [33]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=[col for col in LFP_SPECTRAL_DF if "all_freq" in col], errors="ignore")

In [34]:
LFP_SPECTRAL_DF

,index,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,...,trial_mPFC_power_theta,trial_mPFC_power_gamma,trial_MD_power_theta,trial_MD_power_gamma,trial_LH_power_theta,trial_LH_power_gamma,trial_BLA_power_theta,trial_BLA_power_gamma,trial_vHPC_power_theta,trial_vHPC_power_gamma
0,0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,...,0.030160,0.002167,0.038922,0.001269,0.040500,0.001394,0.034031,0.002251,0.021546,0.002843
1,1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,...,0.028266,0.002177,0.045080,0.001082,0.049413,0.001238,0.039506,0.002506,0.035230,0.002768
2,2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,...,0.032059,0.002143,0.047931,0.001078,0.051984,0.001189,0.039781,0.002418,0.036432,0.002515
3,3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,...,0.035129,0.001958,0.048240,0.001254,0.048795,0.001418,0.044025,0.003139,0.030132,0.002564
4,4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,...,0.024816,0.002066,0.033157,0.001108,0.032718,0.001254,0.026005,0.002629,0.025761,0.003003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,1364,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,60714,60814,60914,2.2,...,0.023956,0.000833,0.017446,0.000899,0.018835,0.001020,0.019226,0.001786,0.021964,0.001045
963,1365,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,62211,62311,62410,2.2,...,0.031187,0.000980,0.029963,0.000808,0.027726,0.000967,0.022851,0.002094,0.030460,0.001304
964,1366,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,64207,64307,64406,2.2,...,0.023314,0.000836,0.036276,0.001013,0.033335,0.001172,0.025945,0.001956,0.032830,0.001386
965,1367,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,66104,66204,66303,2.2,...,0.029101,0.000952,0.028479,0.000766,0.025456,0.000916,0.023135,0.002034,0.027572,0.001236


In [35]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

index
tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
baseline_start_timestamp
baseline_stop_timestamp
baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity
baseline_video_frame
baseline_video_timestamps
trial_agent_location

In [37]:
LFP_SPECTRAL_DF.to_csv("./proc/rce_pilot_2_trial_averages.csv")

## Power plots

In [ ]:
raise ValueError()

In [ ]:
LFP_SPECTRAL_DF["trial_label"]

In [ ]:
LFP_SPECTRAL_DF["experiment"]

In [ ]:
LFP_SPECTRAL_DF["experiment_and_label"] = LFP_SPECTRAL_DF.apply(lambda x: "{}_{}".format(x["experiment"], x["trial_label"]) if x["experiment"] != x["trial_label"] else x["experiment"], axis=1)

In [ ]:
LFP_SPECTRAL_DF["experiment_and_label"] = LFP_SPECTRAL_DF["experiment_and_label"].apply(lambda x: x.replace("omission_rewarded", "alone_rewarded").replace("standard_rewarded", "alone_rewarded").replace("standard", "").strip("_"))

In [ ]:
LFP_SPECTRAL_DF["experiment_and_label"].unique()

In [ ]:
LFP_SPECTRAL_DF["experiment_and_comp"] = LFP_SPECTRAL_DF.apply(lambda x: "{}_{}".format(x["experiment"], x["trial_label"]) if x["experiment"] != x["trial_label"] else x["experiment"], axis=1)

In [ ]:
LFP_SPECTRAL_DF["experiment_and_comp"] = LFP_SPECTRAL_DF["experiment_and_comp"].apply(lambda x: x.replace("omission_rewarded", "alone_rewarded").replace("standard_rewarded", "alone_rewarded").replace("standard", "").strip("_"))

In [ ]:
LFP_SPECTRAL_DF["experiment_and_comp"] = LFP_SPECTRAL_DF.apply(lambda x: "_".join([x["experiment_and_comp"], x["competition_closeness"]]).strip("_"), axis=1)

In [ ]:
LFP_SPECTRAL_DF["experiment_and_comp"].unique()

In [ ]:
LFP_SPECTRAL_DF["experiment_and_comp"].unique()

In [ ]:
LFP_SPECTRAL_DF

In [ ]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

In [ ]:
export_LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.copy()

In [ ]:
export_LFP_SPECTRAL_DF = export_LFP_SPECTRAL_DF.drop(columns = [col for col in export_LFP_SPECTRAL_DF.columns if "trace" in col or "RMS" in col], errors="ignore")
export_LFP_SPECTRAL_DF = export_LFP_SPECTRAL_DF.drop(columns = [col for col in export_LFP_SPECTRAL_DF.columns if "lfp" in col], errors="ignore")
export_LFP_SPECTRAL_DF = export_LFP_SPECTRAL_DF.drop(columns = [col for col in export_LFP_SPECTRAL_DF.columns if "power" in col], errors="ignore")

In [ ]:
export_LFP_SPECTRAL_DF

In [ ]:
columns_to_keep = ['tracked_subject', 'video_name',
'current_subject', 'tone_start_frame', 'reward_start_frame',
'tone_stop_frame', 'condition', 'competition_closeness', 'notes',
'experiment', 'session_dir', 'all_subjects', 'tone_start_timestamp',
'tone_stop_timestamp', 'trial_label', 'cohort', 'tone_frames',
'recording', 'agent',
'baseline_start_timestamp',
'baseline_stop_timestamp', 'experiment_and_comp',
'experiment_and_comp']

In [ ]:
columns_to_keep = sorted(columns_to_keep, key=lambda x: x.split("_")[-1])

In [ ]:
columns_to_keep

In [ ]:
power_columns = set([col.replace("baseline_", "").replace("trial_", "") for col in LFP_SPECTRAL_DF.columns if "all_frequencies" in col])


In [ ]:
power_columns

In [ ]:
all_frequencies_col = [col for col in LFP_SPECTRAL_DF.columns if "all_frequencies" in col]

averaged_LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.copy()
for col in all_frequencies_col:
    averaged_LFP_SPECTRAL_DF[col] = averaged_LFP_SPECTRAL_DF[col].apply(lambda x: np.nanmean(x, axis=0))

In [ ]:
if os.path.exists("./proc/plots/spectra"):
    shutil.rmtree("./proc/plots/spectra")

In [ ]:
Path("./proc/plots/spectra/session_average/competitive/png").mkdir(parents=True, exist_ok=True)
Path("./proc/plots/spectra/session_average/competitive/svg").mkdir(parents=True, exist_ok=True)
Path("./proc/plots/spectra/session_average/trial_type/png").mkdir(parents=True, exist_ok=True)
Path("./proc/plots/spectra/session_average/trial_type/svg").mkdir(parents=True, exist_ok=True)

In [ ]:
comp_to_style = {key: ("--" if "no_comp" in key else "-") for key in averaged_LFP_SPECTRAL_DF["experiment_and_comp"].unique() }

In [ ]:
for col in averaged_LFP_SPECTRAL_DF.columns:
    print(col)

In [ ]:
averaged_LFP_SPECTRAL_DF["trial_label"].unique()

In [ ]:
session_types

In [ ]:
session_df = averaged_LFP_SPECTRAL_DF[averaged_LFP_SPECTRAL_DF["recording"].str.contains("novel")].copy()


In [ ]:
session_types

In [ ]:
averaged_LFP_SPECTRAL_DF

# TODO Get all trials even if they don't have spikes

In [ ]:
grouped_all_trials_df = averaged_LFP_SPECTRAL_DF.groupby(["experiment_and_label", "session_dir", "subject"]).agg({col: stack_arrays for col in all_frequencies_col})

In [ ]:
grouped_all_trials_df = grouped_all_trials_df.reset_index()

In [ ]:
grouped_all_trials_df.head()

In [ ]:
subj_to_color = {"1.1": "red", "1.2": "blue", "1.4": "green"}

In [ ]:
from collections import OrderedDict

In [ ]:
low_freq = 0
high_freq = 51
current_frequencies = range(low_freq, high_freq)

for column in power_columns:
    print(column)
    baseline_column = "baseline_{}".format(column)
    trial_column = "trial_{}".format(column)
    plot_title = "{}".format(column).replace("_all_frequencies", "").replace("_granger", "").replace("_power", "").replace("_coherence", "")
    if "granger" in column:
        plot_title = plot_title.replace("_", " to ")
    elif "power" in column:
        plot_title = plot_title.split("_")[0]

    elif "coherence" in column:
        plot_title = " and ".join(plot_title.split("_")[:2])
    else:
        plot_title = plot_title.replace("_", " and ")

    # plt.title("{} {}".format(preprocessing_type, plot_title))
    
    for trial_label in grouped_all_trials_df["experiment_and_label"].unique():
        current_df = grouped_all_trials_df[grouped_all_trials_df["experiment_and_label"] == trial_label].copy()
    
        ## Setting up the y-axis and the titles
        if "power" not in column:
            pass
        if "trace" in column:
            continue
        fig, ax = plt.subplots()
        

        plt.xlabel("Frequency (Hz)", fontsize=FONTSIZE)

        if "power" in column:
            plt.ylabel("Power (A.U.)", fontsize=FONTSIZE)

        elif "coherence" in column:
            plt.ylabel("Coherence", fontsize=FONTSIZE)

        elif "granger" in column:
            plt.ylabel("Granger", fontsize=FONTSIZE)

        else:
            plt.ylabel("(A.U.)", fontsize=FONTSIZE)


        plt.xlim(low_freq, high_freq-1) 
        plt.xticks(np.arange(low_freq, high_freq-1, 5))

        if "power" in column:
            # plt.ylim(0,0.01)
            # plt.yscale("log")
            # plt.ylim((10**-3.5,10**-0.5))
            ax.set_ylim(auto=True)
            pass
        else:
            # pass
            plt.ylim(0, 1)

        for band, (min_freq, max_freq) in BAND_TO_FREQ_PLOT.items():
            ax.axvspan(min_freq, max_freq, facecolor=BAND_TO_FREQ_COLOR[band], alpha=0.1, label=band)

        for index, row in current_df.iterrows():   
            #### DURING TRIAL ####
            mean_power = np.nanmean(row[trial_column], axis=0)[low_freq: high_freq]
            # print(mean_power)

            # nans, x= nan_helper(mean_power)
            # mean_power[nans]= np.interp(x(nans), x(~nans), mean_power[~nans])
            sem_power = stats.sem(row[trial_column], axis=0, ddof=0)

            # sem_power = np.nanstd(row[trial_column], axis=0) / ((row[trial_column].shape[0])**0.5)
            sem_power = sem_power[low_freq: high_freq]
            # nans, x= nan_helper(sem_power)
            # sem_power[nans]= np.interp(x(nans), x(~nans), sem_power[~nans])   

            baseline_mean_power = np.nanmean(row[baseline_column], axis=0)[low_freq: high_freq]
            # print(mean_power)
            baseline_sem_power = stats.sem(row[baseline_column], axis=0, ddof=0)
            baseline_sem_power = baseline_sem_power[low_freq: high_freq]

            try:
                ax = sns.lineplot(x=current_frequencies, y=mean_power, \
                label=row["subject"], linewidth=1, color=subj_to_color[row["subject"]])

                plt.fill_between(current_frequencies, 
                mean_power - sem_power, mean_power + sem_power, \
                alpha=0.1, color=subj_to_color[row["subject"]])


            except Exception as e: 
                print(e)

        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys(), fontsize=8, ncols=2)

        ax.set_ylim(auto=True)
        plt.tight_layout()
        plt.title("{} {} {}".format(preprocessing_type, plot_title, trial_label))

        column = column.replace("_all_frequencies", "")
        spectra_type = column.split("_")[-1]
        brain_regions = "_".join((column.split("_")[:-1]))

        plt.savefig("./proc/plots/spectra/session_average/trial_type/png/{}_{}hz_{}_{}_{}_spectra.png".format(low_freq, high_freq, spectra_type, plot_title, trial_label))
        plt.savefig("./proc/plots/spectra/session_average/trial_type/svg/{}_{}hz_{}_{}_{}_spectra.svg".format(low_freq, high_freq, spectra_type, plot_title, trial_label))
        plt.show()



# Old code below

In [ ]:
raise ValueError()

In [ ]:
low_freq = 0
high_freq = 51
current_frequencies = range(low_freq, high_freq)

for session in session_types:
    print(session)
    session_df = averaged_LFP_SPECTRAL_DF[averaged_LFP_SPECTRAL_DF["recording"].str.contains(session)].copy()
    
    # Apply the function to each group and each column
    grouped_all_trials_df = session_df.groupby(["experiment_and_label"]).agg({col: stack_arrays for col in all_frequencies_col})

    grouped_all_trials_df = pd.DataFrame(grouped_all_trials_df).reset_index()    
    
    for column in power_columns:
        if "power" not in column:
            pass
        if "trace" in column:
            continue
        fig, ax = plt.subplots()
        print(column)
        baseline_column = "baseline_{}".format(column)
        trial_column = "trial_{}".format(column)
        plot_title = "{}".format(column).replace("_all_frequencies", "").replace("_granger", "").replace("_power", "").replace("_coherence", "")
        if "granger" in column:
            plot_title = plot_title.replace("_", " to ")
        elif "power" in column:
            plot_title = plot_title.split("_")[0]

        elif "coherence" in column:
            plot_title = " and ".join(plot_title.split("_")[:2])
        else:
            plot_title = plot_title.replace("_", " and ")

        plt.title("{} {}".format(preprocessing_type, plot_title))
        
        plt.xlabel("Frequency (Hz)", fontsize=FONTSIZE)
        
        if "power" in column:
            plt.ylabel("Power (A.U.)", fontsize=FONTSIZE)

        elif "coherence" in column:
            plt.ylabel("Coherence", fontsize=FONTSIZE)
        
        elif "granger" in column:
            plt.ylabel("Granger", fontsize=FONTSIZE)

        else:
            plt.ylabel("(A.U.)", fontsize=FONTSIZE)

        
        plt.xlim(low_freq, high_freq-1) 
        plt.xticks(np.arange(low_freq, high_freq-1, 5))

        for band, (min_freq, max_freq) in BAND_TO_FREQ_PLOT.items():
            ax.axvspan(min_freq, max_freq, facecolor=BAND_TO_FREQ_COLOR[band], alpha=0.1, label=band)
        
        if "power" in column:
            # plt.ylim(0,0.01)
            # plt.yscale("log")
            # plt.ylim((10**-3.5,10**-0.5))
            ax.set_ylim(auto=True)
            pass
        else:
            # pass
            plt.ylim(0, 1)
            
        for index, row in grouped_all_trials_df.iterrows():   
            #### DURING TRIAL ####
            mean_power = np.nanmean(row[trial_column], axis=0)[low_freq: high_freq]
            print(mean_power)
            
            # nans, x= nan_helper(mean_power)
            # mean_power[nans]= np.interp(x(nans), x(~nans), mean_power[~nans])
            sem_power = stats.sem(row[trial_column], axis=0, ddof=0)
            
            # sem_power = np.nanstd(row[trial_column], axis=0) / ((row[trial_column].shape[0])**0.5)
            sem_power = sem_power[low_freq: high_freq]
            # nans, x= nan_helper(sem_power)
            # sem_power[nans]= np.interp(x(nans), x(~nans), sem_power[~nans])   
            
            
            baseline_mean_power = np.nanmean(row[baseline_column], axis=0)[low_freq: high_freq]
            print(mean_power)
            
            baseline_sem_power = stats.sem(row[baseline_column], axis=0, ddof=0)
            baseline_sem_power = baseline_sem_power[low_freq: high_freq]

            try:
                ax = sns.lineplot(x=current_frequencies, y=mean_power, \
                label="trial "+ " ".join("{}".format(row["experiment_and_label"]).split("_")), linewidth=3, color=OUTCOME_TO_COLOR[row["experiment_and_label"]])

                plt.fill_between(current_frequencies, 
                mean_power - sem_power, mean_power + sem_power, \
                alpha=0.1, color=OUTCOME_TO_COLOR[row["experiment_and_label"]])
                
#                 ax = sns.lineplot(x=current_frequencies, y=baseline_mean_power, \
#                 label="baseline " + " ".join("{}".format(row["experiment_and_label"]).split("_")), linewidth=3, linestyle="--", color=OUTCOME_TO_COLOR[row["experiment_and_label"]])

#                 plt.fill_between(current_frequencies, 
#                 baseline_mean_power - baseline_sem_power, baseline_mean_power + baseline_sem_power, \
#                 alpha=0.1, color=OUTCOME_TO_COLOR[row["experiment_and_label"]])
            
            
            
            
            except Exception as e: 
                print(e)
        # plt.legend(fontsize=10)
        plt.legend(fontsize=8, loc="lower center", ncols=2)

        ax.set_ylim(auto=True)
        plt.tight_layout()
        
        column = column.replace("_all_frequencies", "")
        spectra_type = column.split("_")[-1]
        brain_regions = "_".join((column.split("_")[:-1]))
        
        plt.savefig("./proc/plots/spectra/session_average/trial_type/png/{}_{}hz_{}_{}_{}_spectra.png".format(low_freq, high_freq, spectra_type, session, plot_title))
        plt.savefig("./proc/plots/spectra/session_average/trial_type/svg/{}_{}hz_{}_{}_{}_spectra.svg".format(low_freq, high_freq, spectra_type, session, plot_title))
        plt.show()
        
    

In [ ]:
averaged_LFP_SPECTRAL_DF["experiment_and_comp"].unique()

In [ ]:
comp_labels = ['win_competitive', 'lose_no_comp', 'lose_competitive',
       'win_no_comp', 'alone_rewarded']

In [ ]:
low_freq = 0
high_freq = 51
current_frequencies = range(low_freq, high_freq)


session_df = averaged_LFP_SPECTRAL_DF[averaged_LFP_SPECTRAL_DF["experiment_and_comp"].isin(comp_labels)].copy()

# Apply the function to each group and each column
grouped_all_trials_df = session_df.groupby("experiment_and_comp").agg({col: stack_arrays for col in all_frequencies_col})

grouped_all_trials_df = pd.DataFrame(grouped_all_trials_df).reset_index()    

for column in power_columns:
    if "power" not in column:
        pass
    if "trace" in column:
        continue
    fig, ax = plt.subplots()
    print(column)
    baseline_column = "baseline_{}".format(column)
    trial_column = "trial_{}".format(column)
    plot_title = "{}".format(column).replace("_all_frequencies", "").replace("_granger", "").replace("_power", "").replace("_coherence", "")
    if "granger" in column:
        plot_title = plot_title.replace("_", " to ")
    elif "power" in column:
        plot_title = plot_title.split("_")[0]

    elif "coherence" in column:
        plot_title = " and ".join(plot_title.split("_")[:2])
    else:
        plot_title = plot_title.replace("_", " and ")

    plt.title("{} {}".format(preprocessing_type, plot_title))

    plt.xlabel("Frequency (Hz)", fontsize=FONTSIZE)

    if "power" in column:
        plt.ylabel("Power (A.U.)", fontsize=FONTSIZE)

    elif "coherence" in column:
        plt.ylabel("Coherence", fontsize=FONTSIZE)

    elif "granger" in column:
        plt.ylabel("Granger", fontsize=FONTSIZE)

    else:
        plt.ylabel("(A.U.)", fontsize=FONTSIZE)


    plt.xlim(low_freq, high_freq-1) 
    plt.xticks(np.arange(low_freq, high_freq-1, 5))

    for band, (min_freq, max_freq) in BAND_TO_FREQ_PLOT.items():
        ax.axvspan(min_freq, max_freq, facecolor=BAND_TO_FREQ_COLOR[band], alpha=0.1, label=band)

    if "power" in column:
        # plt.ylim(0,0.01)
        # plt.yscale("log")
        # plt.ylim((10**-3.5,10**-0.5))
        ax.set_ylim(auto=True)
        pass
    else:
        # pass
        plt.ylim(0, 1)

    for index, row in grouped_all_trials_df.iterrows():   
        #### DURING TRIAL ####
        mean_power = np.nanmean(row[trial_column], axis=0)[low_freq: high_freq]
        print(mean_power)

        # nans, x= nan_helper(mean_power)
        # mean_power[nans]= np.interp(x(nans), x(~nans), mean_power[~nans])
        sem_power = stats.sem(row[trial_column], axis=0, ddof=0)

        # sem_power = np.nanstd(row[trial_column], axis=0) / ((row[trial_column].shape[0])**0.5)
        sem_power = sem_power[low_freq: high_freq]
        # nans, x= nan_helper(sem_power)
        # sem_power[nans]= np.interp(x(nans), x(~nans), sem_power[~nans])   


        baseline_mean_power = np.nanmean(row[baseline_column], axis=0)[low_freq: high_freq]
        print(mean_power)

        baseline_sem_power = stats.sem(row[baseline_column], axis=0, ddof=0)
        baseline_sem_power = baseline_sem_power[low_freq: high_freq]

        try:
            ax = sns.lineplot(x=current_frequencies, y=mean_power, \
            label=""+ " ".join("{}".format(row["experiment_and_comp"]).split("_")), linewidth=3, linestyle=comp_to_style[row["experiment_and_comp"]], color=OUTCOME_TO_COLOR[row["experiment_and_comp"]])

            plt.fill_between(current_frequencies, 
            mean_power - sem_power, mean_power + sem_power, \
            alpha=0.1, color=OUTCOME_TO_COLOR[row["experiment_and_comp"]])

#             ax = sns.lineplot(x=current_frequencies, y=baseline_mean_power, \
#             label="baseline " + " ".join("{}".format(row["experiment_and_comp"]).split("_")), linewidth=3, linestyle="--", color=OUTCOME_TO_COLOR[row["experiment_and_comp"]])

#             plt.fill_between(current_frequencies, 
#             baseline_mean_power - baseline_sem_power, baseline_mean_power + baseline_sem_power, \
#             alpha=0.1, color=OUTCOME_TO_COLOR[row["experiment_and_comp"]])




        except Exception as e: 
            print(e)
    # plt.legend(fontsize=10)
    plt.legend(fontsize=8, loc="upper right", ncols=2)

    ax.set_ylim(auto=True)
    plt.tight_layout()

    column = column.replace("_all_frequencies", "")
    spectra_type = column.split("_")[-1]
    brain_regions = "_".join((column.split("_")[:-1]))

    plt.savefig("./proc/plots/spectra/session_average/competitive/png/{}_{}hz_{}_{}_spectra.png".format(low_freq, high_freq, spectra_type, plot_title))
    plt.savefig("./proc/plots/spectra/session_average/competitive/svg/{}_{}hz_{}_{}_spectra.svg".format(low_freq, high_freq, spectra_type, plot_title))
    plt.show()



In [ ]:
raise ValueError()

In [ ]:
Path("./proc/plots/spectra/subj/png").mkdir(parents=True, exist_ok=True)


In [ ]:
low_freq = 0
high_freq = 51
current_frequencies = range(low_freq, high_freq)

# Iterating through each power column
for column in power_columns:
    if "power" not in column:
        pass
    if "trace" in column:
        continue

    print(column)
    baseline_column = "baseline_{}".format(column)
    trial_column = "trial_{}".format(column)
    
    plot_title = "{}".format(column).replace("_all_frequencies", "").replace("_granger", "").replace("_power", "").replace("_coherence", "")
    if "granger" in column:
        plot_title = plot_title.replace("_", " to ")
    elif "power" in column:
        pass
    elif "coherence" in column:
        plot_title = " and ".join(plot_title.split("_")[:2])

    else:
        plot_title = plot_title.replace("_", " and ")
    
    session_max = averaged_LFP_SPECTRAL_DF[trial_column].apply(lambda x: np.median(x[low_freq:high_freq])).max() * 10
    print(session_max)
    
    column = column.replace("_all_frequencies", "")
    spectra_type = column.split("_")[-1]
    brain_regions = "_".join((column.split("_")[:-1]))
    
    for session in session_types:
        session_df = averaged_LFP_SPECTRAL_DF[averaged_LFP_SPECTRAL_DF["recording"].str.contains(session)].copy()
        
        
        
        for subj in session_df["current_subject"].unique():
            subj_df = session_df[session_df["current_subject"] == subj].copy()

            # Apply the function to each group and each column
            grouped_all_trials_df = subj_df.groupby('experiment_and_comp').agg({col: stack_arrays for col in [baseline_column, trial_column]})
            grouped_all_trials_df = pd.DataFrame(grouped_all_trials_df).reset_index()
            
            fig, ax = plt.subplots()
            
            plt.title("{} {} {} LFP".format(preprocessing_type, plot_title, subj))
            plt.xlabel("Frequency (Hz)", fontsize=FONTSIZE)

            if "power" in column:
                plt.ylabel("Power (A.U.)", fontsize=FONTSIZE)
            elif "coherence" in column:
                plt.ylabel("Coherence", fontsize=FONTSIZE)
            elif "granger" in column:
                plt.ylabel("Granger", fontsize=FONTSIZE)
            else:
                plt.ylabel("(A.U.)", fontsize=FONTSIZE)
                
            plt.xlim(low_freq, high_freq-1) 
            plt.xticks(np.arange(low_freq, high_freq-1, 5))
            
            # Shading the frequency bands
            for band, (min_freq, max_freq) in BAND_TO_FREQ_PLOT.items():
                ax.axvspan(min_freq, max_freq, facecolor=BAND_TO_FREQ_COLOR[band], alpha=0.1, label=band)
            
            if "power" in column:
                # plt.ylim(0, session_max)
                # plt.ylim(0, brain_region_to_ylim[brain_regions])
                pass
            else:
                plt.ylim(0, 1)
            
            for index, row in grouped_all_trials_df.iterrows():   
                #### DURING TRIAL ####
                mean_power = np.nanmean(row[trial_column], axis=0, dtype=float)[low_freq: high_freq]
                sem_power = np.nanstd(row[trial_column], axis=0, dtype=float) / ((row[trial_column].shape[0])**0.5)
                sem_power = sem_power[low_freq: high_freq]

                try:
                    ax = sns.lineplot(x=current_frequencies, y=mean_power, \
                    label=" ".join("{}".format(row["experiment_and_comp"]).split("_")), linewidth=3, linestyle=comp_to_style[row["experiment_and_comp"]],  color=OUTCOME_TO_COLOR[row["experiment_and_comp"]])

                    plt.fill_between(current_frequencies, 
                    mean_power - sem_power, mean_power + sem_power, \
                    alpha=0.1, color=OUTCOME_TO_COLOR[row["experiment_and_comp"]])
                except Exception as e: 
                    print(e)
            plt.legend(fontsize=8, loc="upper right", ncols=2)
            plt.tight_layout()


            plt.savefig("./proc/plots/spectra/subj/png/{}_{}hz_{}_{}_{}_{}_spectra.png".format(low_freq, high_freq, spectra_type, subj, session, plot_title))
            # plt.savefig("./proc/plots/spectra/subj/svg/{}_{}_{}_{}_{}_spectra.svg".format(preprocessing_type, spectra_type, brain_regions, session, subj))
            plt.show()

In [ ]:
raise ValueError()

In [ ]:
export_LFP_SPECTRAL_DF = export_LFP_SPECTRAL_DF[columns_to_keep].drop_duplicates(subset=['recording', 'tone_start_timestamp'])

In [ ]:
recording_dict = defaultdict(dict)
for recording in export_LFP_SPECTRAL_DF["recording"].unique():
    recording_dict[recording] = {}
    for label in export_LFP_SPECTRAL_DF["experiment_and_comp"].unique():
        current_df = export_LFP_SPECTRAL_DF[(export_LFP_SPECTRAL_DF["recording"] == recording) & (export_LFP_SPECTRAL_DF["experiment_and_comp"] == label)].copy()
        current_list = current_df.apply(lambda row: np.array([row["tone_start_timestamp"]//20, row["tone_stop_timestamp"]//20]).astype(int), axis=1).to_list()
        if current_list:
            recording_dict[recording][label] = np.vstack(current_list)
        else: 
            recording_dict[recording][label] = np.array([]).astype(int)

In [ ]:
import pickle

with open("./proc/rce2_trial_timestamps.pkl", 'wb') as handle:
    pickle.dump(recording_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
recording_dict = defaultdict(dict)
for recording in export_LFP_SPECTRAL_DF["recording"].unique():
    recording_dict[recording] = {}
    for label in export_LFP_SPECTRAL_DF["experiment_and_comp"].unique():
        current_df = export_LFP_SPECTRAL_DF[(export_LFP_SPECTRAL_DF["recording"] == recording) & (export_LFP_SPECTRAL_DF["experiment_and_comp"] == label)].copy()
        current_list = current_df.apply(lambda row: np.array([row["tone_start_timestamp"]//20, row["tone_stop_timestamp"]//20]).astype(int), axis=1).to_list()
        if current_list:
            recording_dict[recording][label] = np.vstack(current_list)
        else: 
            recording_dict[recording][label] = np.array([]).astype(int)

In [ ]:
recording_dict

In [ ]:
import pickle

with open("./proc/rce2_competitive_trial_timestamps.pkl", 'wb') as handle:
    pickle.dump(recording_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)